# Creation of Star Map

This notebook should handle the creation of the star map

## Importing necessary packages

In [1]:
from astropy.coordinates import EarthLocation, AltAz, get_sun, get_body, SkyCoord
from astropy.time import Time
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import pandas as pd
import ast
from pathlib import Path
import os
from matplotlib.backends.backend_agg import FigureCanvasAgg

## Setting up necessary directories

After running the next cell, folders will be created on the path listed in `Resources_Path.txt`. The format of the directory inside the text file should be `DriveLetter:\Path`.

In this path a `Photo Related Files` folder will be created. This folder will hold the other content such as the background image and the generated graphics. The only thing that is needed to be placed here are the background images as everything else will be generated by cells found in this notebook. 

In [2]:
working_dir = Path.cwd()
os.chdir(working_dir)

with open("Resources_Path.txt", "r") as resources_text:
    resources_dir = Path(resources_text.readline())

workspace_dir = resources_dir / "Photo Related Files"

if not workspace_dir.exists():
    os.mkdir(workspace_dir)

data_process_dir = working_dir / "Data Processed"
connection_dir = data_process_dir / "Constellation Connection"
clean_star_list_dir = data_process_dir / "Clean_Star_List.csv"
constellation_star_position_list_dir = data_process_dir / "Constellation_Star_Position_List.csv"

## Importing Data

The following cells will just import the processed data

### Stars Info

In [3]:
clean_frame = pd.read_csv(clean_star_list_dir)

star_list = []
for index, row in clean_frame.iterrows():
    star_RA = row["RA"]
    star_Dec = row["Dec"]
    star_VMag = row["Vmag"]
    star_PMag = row["Pmag"]
    star_alpha = row["AlphaPlot"]
    value_list = [star_RA, star_Dec, star_VMag, star_PMag, star_alpha]

    star_list.append(value_list)

star_position_frame = pd.read_csv(constellation_star_position_list_dir)

star_position_list = []
for index, row in star_position_frame.iterrows():
    star_RA = row["RA"]
    star_Dec = row["Dec"]

    value_list = [star_RA, star_Dec]
    star_position_list.append(value_list)

### Connection info

In [4]:
outer_connection_list = []
for connection_file in connection_dir.iterdir():
    connection_frame = pd.read_csv(connection_file)

    connection_list = []

    for index, row in connection_frame.iterrows():
        RA_points = np.array(ast.literal_eval(row["RA Points"]))
        Dec_points = np.array(ast.literal_eval(row["Dec Points"]))

        combination_pack = np.array([RA_points, Dec_points])

        connection_list.append(combination_pack)

    connection_count = 0
    outer_connection_list.append(connection_list)

### More stars info

In [5]:
count = 0
ra_list = []
dec_list = []
size_list = []
alpha_list = []

star_amount = 500

while count < star_amount:
    current_star = star_list[count]
    ra_list.append(current_star[0])
    dec_list.append(current_star[1])
    size_list.append(current_star[3])
    alpha_list.append(current_star[4])
    count += 1

np_ra_list = np.array(ra_list)
np_dec_list = np.array(dec_list)


count = 0

pos_ra_list = []
pos_dec_list = []
while count < len(star_position_list):
    pos_ra_list.append(star_position_list[count][0])
    pos_dec_list.append(star_position_list[count][1])

    count += 1

np_pos_ra = np.array(pos_ra_list)
np_pos_dec = np.array(pos_dec_list)

## Defining some functions

The next cell defines the important functions used in plotting. The functions defined in the next cell functions the same, which is just stereographic projection.

In [6]:
def altaz_to_stereographic(alt, az):

    # Converting altitude and azimuth to radian
    # Altitude is taken as negative since projection comes from the south pole
    alt_rad = -alt.to(u.rad)
    az_rad = az.to(u.rad)

    # Convert altitude and azimuth to Cartesian coordinates
    x = np.cos(alt_rad) * np.sin(az_rad)
    y = np.cos(alt_rad) * np.cos(az_rad)
    z = np.sin(alt_rad)

    # Project onto the 2D plane (stereographic projection)
    rho = 1 / (1 - z)
    x_proj = rho * x
    y_proj = rho * y

    return x_proj, y_proj


def altaz_to_stereographic_calculate(alt, az):

    # Convert altitude and azimuth to Cartesian coordinates
    x = np.cos(np.radians(-alt)) * np.sin(np.radians(az))
    y = np.cos(np.radians(-alt)) * np.cos(np.radians(az))
    z = np.sin(np.radians(-alt))

    # Project onto the 2D plane (stereographic projection)
    rho = 1 / (1 - z)
    x_proj = rho * x
    y_proj = rho * y

    return x_proj, y_proj

## Setting up the conditions

In the following cells, the specific condition such as the location and time of the observation will be set

### Location
---
Just input latitude and longitude

In [7]:
latitude = 0
longitude = 0

observer_location = EarthLocation(lat=latitude, lon=longitude)

### Time
---
Time of observation. The notebook can use the time of the machine but I can also choose a specific time by changing the `iso_time` variable. If the specific time will be used, the last line in the cell should be uncommented. It should be noted that `iso_time` is UTC+0

In [8]:
date_time = Time.now()
plot_time = date_time

iso_time = "2000-01-01T12:00:00"  # YYYY-MM-DDTHH:MM:SS
specific_time = Time(iso_time, format='isot', scale='utc')
plot_time = specific_time

## Creation of Map Resources

In the next cells, the necessary resources to create the map will be created.

### Equator and Ecliptic
---
The following cell would generate the `SkyCoord` objects for the celestial equator and the ecliptic.

In [9]:
longitude = np.linspace(0, 360, 360+1) * u.deg
latitude = 0 * u.deg

# Equator is computed here
equator_coords = SkyCoord(ra=longitude, dec=latitude, unit='deg', frame='icrs')

# Ecliptic is computed here
ecliptic_coords = SkyCoord(lon=longitude, lat=latitude,
                           unit='deg', frame='barycentrictrueecliptic')

### Creation of AltAz Frame
---
The AltAz frame essentially reorients the celestial sphere to align with the position and time of the observer. It encompasses two values for an object: altitude and azimuth. These two parameters function analogously to right ascension and declination.

The celestial equator and ecliptic points are projected onto their respective AltAz locations. Additionally, the positions of the sun, moon, and planets are determined. The analemma represents the sun's path across the sky at the same time each day throughout the year, while the day path illustrates the sun's trajectory over the course of a day. Both paths of the sun are also projected onto their AltAz locations.


In [10]:
# AltAz frame is set here
AltAz_frame = AltAz(location=observer_location)

# Equator and ecliptic converted to AltAz
equator_coord_altaz = equator_coords.transform_to(
    AltAz(location=observer_location, obstime=plot_time))
ecliptic_coord_altaz = ecliptic_coords.transform_to(
    AltAz(location=observer_location, obstime=plot_time))

# Getting the position of the sun, moon, and the eight planets
sun_position = get_sun(plot_time).transform_to(AltAz_frame)
moon_position = get_body("moon", plot_time).transform_to(AltAz_frame)
mercury_position = get_body("mercury", plot_time).transform_to(AltAz_frame)
venus_position = get_body("venus", plot_time).transform_to(AltAz_frame)
mars_position = get_body("mars", plot_time).transform_to(AltAz_frame)
jupiter_position = get_body("jupiter", plot_time).transform_to(AltAz_frame)
saturn_position = get_body("saturn", plot_time).transform_to(AltAz_frame)
uranus_position = get_body("uranus", plot_time).transform_to(AltAz_frame)
neptune_position = get_body("neptune", plot_time).transform_to(AltAz_frame)

# Creating time entries for future
year_forward = plot_time + np.linspace(0, 365, 365+1)*24*u.hour
day_forward = plot_time + np.linspace(0, 24, 96+1)*u.hour

# Acquiring position of the analemma and the path of the sun
analemma_position = get_sun(year_forward).transform_to(AltAz_frame)
day_path_position = get_sun(day_forward).transform_to(AltAz_frame)

### Miscellaneous Points
---
Several useful points are created in the next cell

In [11]:
azimuth_points = np.linspace(0, 360, 1440+1)

x_proj_0, y_proj_0 = altaz_to_stereographic_calculate(0, azimuth_points)
x_proj_15, y_proj_15 = altaz_to_stereographic_calculate(15, azimuth_points)
x_proj_30, y_proj_30 = altaz_to_stereographic_calculate(30, azimuth_points)
x_proj_45, y_proj_45 = altaz_to_stereographic_calculate(45, azimuth_points)
x_proj_60, y_proj_60 = altaz_to_stereographic_calculate(60, azimuth_points)
x_proj_75, y_proj_75 = altaz_to_stereographic_calculate(75, azimuth_points)

# Semicircle points
x_semicircle = np.linspace(-1, 1, 512+1)
y_circle_above = np.sqrt(1-(x_semicircle**2))
y_circle_below = -np.sqrt(1-(x_semicircle**2))

# Outer ring points
angle_position = np.linspace(0, 2*np.pi, 1440+1)
x_outer_ring = np.cos(angle_position)
y_outer_ring = np.sin(angle_position)

### Mask points
---
The points for the circular mask are created here

In [12]:
count = 0

top_semicircle_vertices = [[-1.25, 1.25], [-1.25, 0]]
bottom_semicircle_vertices = [[-1.25, -1.25], [-1.25, 0]]

while count < len(x_semicircle):

    top_list = [x_semicircle[count], y_circle_above[count]]
    bot_list = [x_semicircle[count], y_circle_below[count]]

    top_semicircle_vertices.append(top_list)
    bottom_semicircle_vertices.append(bot_list)

    count += 1

top_semicircle_vertices += [[1.25, 0], [1.25, 1.25]]
bottom_semicircle_vertices += [[1.25, 0], [1.25, -1.25]]

top_array = np.array(top_semicircle_vertices)
bottom_array = np.array(bottom_semicircle_vertices)

square_vertices = np.array(
    [[1.25, 1.25], [-1.25, 1.25], [-1.25, -1.25], [1.25, -1.25]])

## Actual Resource Creation

In the next cells, the actual image files of the different resources will be created.

### Background Mask

In [13]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

ax.fill(square_vertices[:, 0], square_vertices[:, 1], color='#FFFFFF', alpha=1)
ax.fill(top_array[:, 0], top_array[:, 1], color=(0, 0, 0), alpha=1)
ax.fill(bottom_array[:, 0], bottom_array[:, 1], color=(0, 0, 0), alpha=1)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Background_Mask.png"
PIL_image.save(output_path)
plt.close(fig)

### Outer Background

In [14]:
background_mask_img_path = workspace_dir / "Background_Mask.png"
out_background_mask_img_path = workspace_dir / "Background_01.jpg"

background_mask_img = Image.open(str(background_mask_img_path)).convert("L")
background_mask_img = ImageOps.invert(background_mask_img)

out_background_mask_img = Image.open(
    str(out_background_mask_img_path)).convert("RGBA")
out_background_mask_img = out_background_mask_img.resize((background_mask_img.size[0], background_mask_img.size[1]), Image.LANCZOS)

result_image = Image.new("RGBA", out_background_mask_img.size, (0, 0, 0, 0))
result_image.paste(out_background_mask_img, mask=background_mask_img)

output_path = workspace_dir / "Background_Image.png"
result_image.save(str(output_path))

### Solar System Objects Stereographic Projection

In [15]:
# For the sun, moon, and the eight planets
x_sun, y_sun = altaz_to_stereographic(sun_position.alt, sun_position.az)
x_moon, y_moon = altaz_to_stereographic(moon_position.alt, moon_position.az)
x_mercury, y_mercury = altaz_to_stereographic(
    mercury_position.alt, mercury_position.az)
x_venus, y_venus = altaz_to_stereographic(
    venus_position.alt, venus_position.az)
x_mars, y_mars = altaz_to_stereographic(mars_position.alt, mars_position.az)
x_jupiter, y_jupiter = altaz_to_stereographic(
    jupiter_position.alt, jupiter_position.az)
x_saturn, y_saturn = altaz_to_stereographic(
    saturn_position.alt, saturn_position.az)
x_uranus, y_uranus = altaz_to_stereographic(
    uranus_position.alt, uranus_position.az)
x_neptune, y_neptune = altaz_to_stereographic(
    neptune_position.alt, neptune_position.az)

# For the equator and ecliptic
equator_x, equator_y = altaz_to_stereographic(
    equator_coord_altaz.alt, equator_coord_altaz.az)
ecliptic_x, ecliptic_y = altaz_to_stereographic(
    ecliptic_coord_altaz.alt, ecliptic_coord_altaz.az)

# For the analemma and path of the sun
analemma_x, analemma_y = altaz_to_stereographic(
    analemma_position.alt, analemma_position.az)
day_path_x, day_path_y = altaz_to_stereographic(
    day_path_position.alt, day_path_position.az)

### Outer Rim

In [16]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

ax.plot(1.08*x_outer_ring, 1.08*y_outer_ring, color=(1, 1, 1),linewidth=0.5)
ax.scatter(1.05*x_outer_ring[::45], 1.05*y_outer_ring[::45], color=(1, 1, 1),s=1)
ax.plot(1.02*x_outer_ring, 1.02*y_outer_ring, color=(1, 1, 1),linewidth=0.5)
ax.plot(x_proj_0, y_proj_0, color=(1, 1, 1), linewidth=2.5)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Rim.png"
PIL_image.save(output_path)
plt.close(fig)

### Rings Stereographic Projection

In [17]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

ax.scatter(x_proj_15[::90][:-1], y_proj_15[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_30[::90][:-1], y_proj_30[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_45[::90][:-1], y_proj_45[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_60[::90][:-1], y_proj_60[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(x_proj_75[::90][:-1], y_proj_75[::90][:-1], color=(0.25, 0.5, 1), alpha=0.25, s=0.5)
ax.scatter(0, 0, color=(0.25, 0.5, 1), s=0.5, alpha=0.25)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Rings.png"
PIL_image.save(output_path)
plt.close(fig)

### Paths Stereographic Projection

In [18]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)


# Plotting the equator and ecliptic
ax.plot(equator_x, equator_y, linestyle='--', dashes=(30, 20),
        color=(0.75, 0.5, 1), linewidth=0.25)
ax.plot(ecliptic_x, ecliptic_y, linestyle='--',
        dashes=(30, 20), color=(0.25, 1, 0.5), linewidth=0.25)

# Plotting the sun day path and analemma
ax.plot(analemma_x, analemma_y, linestyle='--', dashes=(15, 10),
        color=(0.5, 0.25, 0.125), linewidth=0.25)
ax.plot(day_path_x, day_path_y, linestyle='--',
        dashes=(30, 20), color=(1, 0.5, 0.25), linewidth=0.25)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Paths.png"
PIL_image.save(output_path)
plt.close(fig)

### Constellation Connections Stereographic Projection

In [19]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)


for outer_connection in outer_connection_list:
    for connection in outer_connection:
        line_coords = SkyCoord(
            ra=connection[0], dec=connection[1], unit='deg', frame='icrs')
        line_altaz = line_coords.transform_to(
            AltAz(location=observer_location, obstime=plot_time))

        line_x, line_y = altaz_to_stereographic(line_altaz.alt, line_altaz.az)
        ax.plot(line_x, line_y, color=(1, 1, 1), linewidth=0.75, alpha=0.875)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Connections.png"
PIL_image.save(output_path)
plt.close(fig)

### Star Mask Stereographic Projection

In [20]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

star_coords = SkyCoord(
    ra=np_pos_ra, dec=np_pos_dec, unit='deg', frame='icrs')
star_altaz = star_coords.transform_to(
    AltAz(location=observer_location, obstime=plot_time))

star_x, star_y = altaz_to_stereographic(star_altaz.alt, star_altaz.az)
#ax.scatter(star_x, star_y, marker=".", color=(1, 1, 1), s=size_list, alpha=alpha_list)
ax.scatter(star_x, star_y, marker=".", color=(0, 0, 0),s=2.5)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Stars Mask.png"
PIL_image.save(output_path)
plt.close(fig)

### Constellation Stars Stereographic Projection

In [21]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

star_coords = SkyCoord(
    ra=np_pos_ra, dec=np_pos_dec, unit='deg', frame='icrs')
star_altaz = star_coords.transform_to(
    AltAz(location=observer_location, obstime=plot_time))

star_x, star_y = altaz_to_stereographic(star_altaz.alt, star_altaz.az)
#ax.scatter(star_x, star_y, marker=".", color=(1, 1, 1), s=size_list, alpha=alpha_list)
ax.scatter(star_x, star_y, marker=".", color=(1, 1, 1),s=0.25, alpha=0.5)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Constellation Stars.png"
PIL_image.save(output_path)
plt.close(fig)

### Modified Constellation Connections

In [22]:
star_mask_img_path = workspace_dir / "Stars Mask.png"
connections_img_path = workspace_dir / "Connections.png"

star_mask_img = Image.open(str(star_mask_img_path)).convert("L")

connections_img = Image.open(
    str(connections_img_path)).convert("RGBA")

result_image = Image.new("RGBA", connections_img.size, (0, 0, 0, 0))
result_image.paste(connections_img, mask=star_mask_img)

output_path = workspace_dir / "Connections Cut.png"
result_image.save(str(output_path))

### General Stars Stereographic Projection

In [23]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)

star_coords = SkyCoord(
    ra=np_ra_list, dec=np_dec_list, unit='deg', frame='icrs')
star_altaz = star_coords.transform_to(
    AltAz(location=observer_location, obstime=plot_time))

star_x, star_y = altaz_to_stereographic(star_altaz.alt, star_altaz.az)
#ax.scatter(star_x, star_y, marker=".", color=(1, 1, 1), s=size_list, alpha=alpha_list)
ax.scatter(star_x, star_y, marker=".", color=(1, 1, 1),s=1)

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Stars.png"
PIL_image.save(output_path)
plt.close(fig)

### Solar System Objects Stereographic Projection

In [24]:
fig, ax = plt.subplots()

fig.set_size_inches(10, 10)

dpi = 600  # Set your desired DPI value
fig.set_dpi(dpi)
fig.patch.set_alpha(0)


# Plotting the sun, moon, and the eight planets


ax.scatter(x_neptune, y_neptune, label="Neptune", marker=r"$♆$",color=(0.75, 0, 1))
ax.scatter(x_uranus, y_uranus, label="Uranus", marker=r"$⛢$",color=(0.678, 0.847, 0.902))
ax.scatter(x_saturn, y_saturn, label="Saturn", marker=r"$♄$",color=(0.647, 0.165, 0.165))
ax.scatter(x_jupiter, y_jupiter, label="Jupiter", marker=r"$♃$",color=(0, 0, 1))
ax.scatter(x_mars, y_mars, label="Mars", marker=r"$♂$",color=(1, 0, 0))
ax.scatter(x_venus, y_venus, label="Venus", marker=r"$♀$",color=(0, 1, 0))
ax.scatter(x_mercury, y_mercury, marker=r'$☿$',color=(1, 1, 0))

ax.scatter(x_sun, y_sun, label="Sun", color=(1, 0.843, 0))
ax.scatter(x_moon, y_moon, label="Moon", color=(0.5, 0.5, 0.5), edgecolors=(1, 1, 1))

# Set limits
ax.set_xlim(-1.25, 1.25)
ax.set_ylim(-1.25, 1.25)
ax.grid(False)
# Remove axes and labels
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])

# Remove frame
ax.set_frame_on(False)

fig.subplots_adjust(left=0, right=1, top=1, bottom=0)

canvas = FigureCanvasAgg(fig)
canvas.draw()

image_array = np.array(canvas.buffer_rgba())
PIL_image = Image.fromarray(image_array)

# Saves image
output_path = workspace_dir / "Solar.png"
PIL_image.save(output_path)
plt.close(fig)

## Merging of the generated images

The next cell initialize the paths of the images. Only the `background_img_path` should be modified to change the background on the inside circle of the map. To change the background outside the circle of the map, change the image used in creating `Background_Image.png`.

In [25]:
background_img_path = workspace_dir / "Background_01.jpg"

rings_img_path = workspace_dir / "Rings.png"
paths_img_path = workspace_dir / "Paths.png"
connections_img_path = workspace_dir / "Connections Cut.png"
constellation_stars_img_path = workspace_dir / "Constellation Stars.png"
stars_img_path = workspace_dir / "Stars.png"
solar_img_path = workspace_dir / "Solar.png"
rim_img_path = workspace_dir / "Rim.png"
out_background_img_path = workspace_dir / "Background_Image.png"

Merging is done in the next cell, with the output saved as `Output.png`

In [26]:
out_background_img = Image.open(str(out_background_img_path))

background_img = Image.open(str(background_img_path))
background_img = background_img.resize((background_mask_img.size[0], background_mask_img.size[1]), Image.LANCZOS)

rings_img = Image.open(str(rings_img_path))
paths_img = Image.open(str(paths_img_path))
connections_img = Image.open(str(connections_img_path))
constellation_stars_img = Image.open(str(constellation_stars_img_path))
stars_img = Image.open(str(stars_img_path))
solar_img = Image.open(str(solar_img_path))
rim_img = Image.open(str(rim_img_path))
out_background_img = Image.open(str(out_background_img_path))

background_img = background_img.convert('RGBA')
rings_img = rings_img.convert('RGBA')
paths_img = paths_img.convert('RGBA')
connections_img = connections_img.convert('RGBA')
constellation_stars_img = constellation_stars_img.convert("RGBA")
stars_img = stars_img.convert('RGBA')
solar_img = solar_img.convert('RGBA')
rim_img = rim_img.convert('RGBA')
out_background_img = out_background_img.convert('RGBA')

overlay = Image.alpha_composite(background_img, rings_img)
overlay = Image.alpha_composite(overlay, paths_img)
overlay = Image.alpha_composite(overlay, connections_img)
overlay = Image.alpha_composite(overlay, constellation_stars_img)
overlay = Image.alpha_composite(overlay, stars_img)
overlay = Image.alpha_composite(overlay, solar_img)
overlay = Image.alpha_composite(overlay, out_background_img)
overlay = Image.alpha_composite(overlay, rim_img)

output_path = resources_dir / "Output.png"

overlay.save(str(output_path))

overlay_scaled = overlay.resize((1080, 1080), Image.LANCZOS)

output_path = resources_dir / "Output Scaled.png"
overlay_scaled.save(str(output_path))